# Exploratory Data analysis
_Written by Thomas Niedermayer and Gunnar Sjúrðarson Knudsen, as a conjoined effort for an interdiscplinary project in Data Science._
* Supervisor: Wolfgang Aussenegg
* Co-Supervisor: Sascha Hunold

Purpose of this notebook is to understand the data quality, and the scale of the task. 
As we ran into several issues w.r.t. data quality, we want to get a deeper understanding of how which data is being handled. Example errors:
* TICKERS being used for multiple companies/ISINS, resulting in not knowing which ISIN an insider trading corresponds to
* Missing ISINs; means that no time series data is available
* Missing TICKERS; Means that no insider trades are available.
* For several companies, there are no insider trades registered. These are therefore also filtered out from the analysis notebook. These wouldn't give an error, but removing them speeds up the runtime
* Sometimes there was no return index data available. **Honestly not sure why this didn't break our analaysis. Maybe this was the reason for the `-Inf`?** In any case, they are also removed
Check which data we've already extracted, and define how to handle various situations

## Load libraries

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import time
import datetime
from dateutil.relativedelta import relativedelta
import requests
import locale
from pandas.io.json import json_normalize
import os
from os.path import exists
import sys
import io

from IPython.display import clear_output, display
from tqdm import tqdm

# Load custom libraries
import source.read_tickers_and_isins as URTI

## Define which data to be loaded

In [2]:
# Set flags for what will be handled
NAME = "Knudsen" # "Niedermayer"
#NAME = "Niedermayer"
prepare_and_download = True

# Constants:
## Different Parameters depending on the setting
if NAME == "Knudsen":
    no_https = False
    to_date_name = "DATE/TIME (DS End Date)"
    STOCK_EXCHANGE = "Nasdaq"
    n_input_files = 7
    _ticker = '%5EIXIC'  
elif NAME == "Niedermayer":
    no_https = True
    to_date_name = "DATE/TIME (DS End Date)"
    STOCK_EXCHANGE = "NYSE"
    n_input_files = 4
    _ticker = "%5Enya"
else:
    raise NotImplementedError

In [3]:
DATA_LOCATION = f'data/{NAME}/'
_insider_location = DATA_LOCATION + 'processed/insider/'

## Read in input data

In [4]:
INPUT_FILE = f'input_data/{NAME}/{STOCK_EXCHANGE} Composite 16.3.2022 plus dead firms - {NAME}.xlsx'

In [5]:
data = URTI.read_tickers_and_isins(INPUT_FILE)

Reading tickers


In [6]:
data

,Type,ISIN CODE,LOC OFF. CODE,NAME,DATASTREAM CODE,CUSIP,TICKER SYMBOL,BASE OR ST DATE,DATE/TIME (DS End Date)
0,2606T9,KYG870761080,UG87076108,10X CAPITAL VENTURE ACQUISITION II A,2606T9,G87076108,VCXA,2021-10-05,2022-03-16
1,95118Z,US88025U1097,U88025U109,10X GENOMICS A,95118Z,88025U109,TXG,2019-09-12,2022-03-16
2,9330V3,US68247Q1022,U68247Q102,111 ADR 1:2,9330V3,68247Q102,YI,2018-09-12,2022-03-16
3,99133U,US81807M2052,U81807M205,17 ED.TGP.ADR 1:10,99133U,81807M205,YQ,2020-12-04,2022-03-16
4,9101KM,US68236V1044,U68236V104,180 LIFE SCIENCES,9101KM,68236V104,ATNF,2017-06-27,2022-03-16
...,...,...,...,...,...,...,...,...,...
4073,30358V,US9898171015,U989817101,ZUMIEZ,30358V,989817101,ZUMZ,2005-05-06,2022-03-16
4074,50259R,US98880R1095,U98880R109,ZW DATA ACTION TECHNOLOGIES,50259R,98880R109,CNET,2007-10-15,2022-03-16
4075,2568RT,US98985X1000,U98985X100,ZYMERGEN,2568RT,98985X100,ZY,2021-04-22,2022-03-16
4076,98116P,US98986X1090,U98986X109,ZYNERBA PHARMACEUTICALS,98116P,98986X109,ZYNE,2015-08-05,2022-03-16


## Read in scraped data
We need this, so we can also exclude handling where we didn't get any data from whereever

### Insider trades

In [7]:
# Define dummy placeholders
tickers = []
trade_counts = []
min_filing_date = []
max_filing_date = []
min_trade_date = []
max_trade_date = []
n_distinct_traders = []
n_distinct_trade_types = []

n_p = []
n_s = []
n_s2 = []
n_a = []
n_d = []
n_g = []
n_f = []
n_m = []
n_x = []
n_c = []
n_w = []

# helpers
counter = 0
total_count = len(data['TICKER SYMBOL'])

# Read in scraped files, and do various aggregations
for ticker in data['TICKER SYMBOL']:
    counter = counter +1
    clear_output(wait=True)
    print(f'Handling {counter} of {total_count}. Currently doing: {ticker}')
    
    dat = pd.read_csv(_insider_location + ticker + '.csv', index_col=0, parse_dates=['FilingDate', 'TradeDate'])


    tickers.append(ticker)
    trade_counts.append(dat.shape[0])

    min_filing_date.append(dat['FilingDate'].min())
    max_filing_date.append(dat['FilingDate'].max())

    min_trade_date.append(dat['TradeDate'].min())
    max_trade_date.append(dat['TradeDate'].max())

    n_distinct_traders.append(dat['InsiderName'].nunique())
    n_distinct_trade_types.append(dat['TradeType'].nunique())


    n_p.append(sum(dat['TradeType'] == 'P - Purchase'))
    n_s.append(sum(dat['TradeType'] == 'S - Sale'))
    n_s2.append(sum(dat['TradeType'] == 'S - Sale+OE'))

    n_a.append(sum(dat['TradeType'] == 'A - Grant'))
    n_d.append(sum(dat['TradeType'] == 'D - Sale to Iss') + sum(dat['TradeType'] == 'D - Sale to issuer'))
    n_g.append(sum(dat['TradeType'] == 'G - Gift'))
    n_f.append(sum(dat['TradeType'] == 'F - Tax'))
    n_m.append(sum(dat['TradeType'] == 'M - Option Ex') + sum(dat['TradeType'] == 'M - OptEx'))
    n_x.append(sum(dat['TradeType'] == 'X - Option Ex') + sum(dat['TradeType'] == 'X - OptEx'))
    n_c.append(sum(dat['TradeType'] == 'C - Cnv Deriv') + sum(dat['TradeType'] == 'C - Converted deriv'))
    n_w.append(sum(dat['TradeType'] == 'W - Inherited'))
    
# Collect to a single data frame
scraped_insider_df = pd.DataFrame({'tickers': tickers
                                   , 'trade_count': trade_counts
                                   , 'min_filing_date': min_filing_date
                                   , 'max_filing_date': max_filing_date
                                   , 'min_trade_date': min_trade_date
                                   , 'max_trade_date': max_trade_date
                                   , 'n_distinct_traders': n_distinct_traders
                                   , 'n_distinct_trade_types': n_distinct_trade_types
                                   , 'P - Purchase (count)': n_p
                                   , 'S - Sale (count)': n_s
                                   , 'S - Sale+OE': n_s2
                                   , 'A - Grant (count)': n_a
                                   , 'D - Sale to Iss (count)': n_d
                                   , 'G - Gift (count)': n_g
                                   , 'F - Tax (count)': n_f
                                   , 'M - Option Ex (count)': n_m
                                   , 'X - Option Ex (count)': n_x
                                   , 'C - Cnv Deriv (count)': n_c
                                   , 'W - Inherited (count)': n_w
                   })
scraped_insider_df

Handling 4078 of 4078. Currently doing: ZNGA


,tickers,trade_count,min_filing_date,max_filing_date,min_trade_date,max_trade_date,n_distinct_traders,n_distinct_trade_types,P - Purchase (count),S - Sale (count),S - Sale+OE,A - Grant (count),D - Sale to Iss (count),G - Gift (count),F - Tax (count),M - Option Ex (count),X - Option Ex (count),C - Cnv Deriv (count),W - Inherited (count)
0,VCXA,0,NaT,NaT,NaT,NaT,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TXG,300,2019-09-16 17:26:06,2022-02-24 19:40:46,2019-09-12,2022-02-22,14,7,3,59,102,10,0,7,0,104,0,15,0
2,YI,0,NaT,NaT,NaT,NaT,0,0,0,0,0,0,0,0,0,0,0,0,0
3,YQ,0,NaT,NaT,NaT,NaT,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ATNF,44,2017-06-27 16:23:30,2022-01-18 16:33:50,2017-06-23,2022-01-14,15,4,11,18,0,14,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4073,ZUMZ,495,2005-05-12 17:00:40,2022-03-16 11:26:59,2005-05-11,2022-03-15,26,7,1,103,46,149,0,32,112,52,0,0,0
4074,CNET,84,2011-08-30 16:30:30,2015-08-18 08:04:40,2011-08-26,2015-08-11,8,4,73,2,0,5,4,0,0,0,0,0,0
4075,ZY,25,2021-04-22 20:58:39,2022-03-15 20:33:53,2021-03-05,2022-03-11,9,7,1,8,0,8,1,1,0,2,0,4,0
4076,ZYNE,37,2015-08-10 15:05:19,2022-02-15 16:13:31,2015-08-05,2022-02-14,15,5,16,1,0,17,0,0,0,1,0,2,0


#### Join the data

In [8]:
data = data.join(scraped_insider_df, rsuffix='_given', lsuffix='_insider', how="left")

### Read in market timeseries

In [9]:
# File location (needs cleansing)
DATA_LOCATION_RI = DATA_LOCATION + 'processed/RI_discard/'
_ri_location = DATA_LOCATION_RI

file_locs_ = os.listdir(_ri_location)
file_locs = [_ri_location + f for f in file_locs_]

# Actually read in the company information
companies = []
print("loading return series...")
for file_loc in tqdm(file_locs):
    with open(file_loc, "rb") as f:
        company = pickle.load(f)
    companies.append(company)

loading return series...


100%|██████████| 4077/4077 [00:00<00:00, 4162.98it/s]


In [10]:
isins = []
names = []
tickers = []
start_dates = []
end_dates = []
start_dates_ts = []
end_dates_ts = []
ts_rows = []
for company in companies:
    isins.append(company.isin)
    names.append(company.name)
    tickers.append(company.ticker)
    start_dates.append(company.start_date)
    end_dates.append(company.end_date)
    start_dates_ts.append(company.return_index_df.index.min())
    end_dates_ts.append(company.return_index_df.index.max())
    ts_rows.append(company.return_index_df.shape[0])

# Collect to a single data frame
scraped_ts_df = pd.DataFrame({'isin': isins
                              , 'ts_rows': ts_rows
                              , 'name': names
                              , 'ticker': tickers
                              , 'start_date': start_dates
                              , 'end_date': end_dates
                              , 'start_date_ts': start_dates_ts
                              , 'end_date_ts': end_dates_ts
                             })
scraped_ts_df

,isin,ts_rows,name,ticker,start_date,end_date,start_date_ts,end_date_ts
0,AGP8696W1045,737,SINOVAC BIOTECH,SVA,2003-09-26,2019-02-22,2016-03-21,2019-02-22
1,AU000000ITL3,1157,INTEGRATED MEDIA TECH.,IMTE,2017-08-11,2022-03-16,2017-08-11,2022-03-16
2,AU0000185993,82,IRIS ENERGY PTY,IREN,2021-11-17,2022-03-16,2021-11-17,2022-03-16
3,AU0000198582,942,CENNTRO ELECTRIC GROUP,CENN,2018-06-20,2022-03-16,2018-06-20,2022-03-16
4,AU0000205205,242,TRITIUM DCFC,DCFC,2021-04-01,2022-03-16,2021-04-01,2022-03-16
...,...,...,...,...,...,...,...,...
4072,VGG870841027,1129,TDH HOLDINGS,PETZ,2017-09-21,2022-03-16,2017-09-21,2022-03-16
4073,VGG9320Z1099,40,VAHANNA TECH EDGE ACQUISITION I A,VHNA,2022-01-13,2022-03-11,2022-01-13,2022-03-11
4074,VGG941841014,727,WAH FU EDUCATION GROUP,WAFU,2019-04-30,2022-03-16,2019-04-30,2022-03-16
4075,VGG9604C1077,315,MEIWU TECHNOLOGY COMPANY,WNW,2020-12-15,2022-03-16,2020-12-15,2022-03-16


#### Join the data

In [11]:
data = data.join(scraped_ts_df, rsuffix='_given', lsuffix='_ts', how="left")

### Start filtering

In [12]:
data['reason_to_exclude'] = 'None'

#### Remove Companies without ISINs

In [13]:
mask = data['ISIN CODE'] == 'NA'
#data.loc[mask, 'reason_to_exclude'] = 'NA ticker'
data.loc[mask, 'reason_to_exclude'] = 'Missing ISIN'
data.loc[mask].shape

(2, 37)

#### Remove companies without trades

In [14]:
mask = data['trade_count'] == 0
#data.loc[mask, 'reason_to_exclude'] = 'NA ticker'
data.loc[mask, 'reason_to_exclude'] = 'No trades done'
data.loc[mask].shape

(922, 37)

#### Remove companies without timeseries

In [15]:
mask = data['ts_rows'].isnull()
data.loc[mask, 'reason_to_exclude'] = 'No timeseries data'
data.loc[mask].shape

(1, 37)

#### Find non-unique tickers

In [16]:
dublicate_tickers = data[data.duplicated(subset=['TICKER SYMBOL'],keep=False)]['TICKER SYMBOL']
dublicate_tickers_mask = data['TICKER SYMBOL'].isin(dublicate_tickers)
data.loc[dublicate_tickers_mask, 'reason_to_exclude'] = 'Non-unique-ticker'
data.loc[dublicate_tickers_mask].shape

(251, 37)

#### Find NA tickers

In [17]:
mask = data['TICKER SYMBOL'] == 'NA'
data.loc[mask, 'reason_to_exclude'] = 'NA ticker'
data.loc[mask].shape

(158, 37)

### Show what we have

In [18]:
data

,Type,ISIN CODE,LOC OFF. CODE,NAME,DATASTREAM CODE,CUSIP,TICKER SYMBOL,BASE OR ST DATE,DATE/TIME (DS End Date),tickers,...,W - Inherited (count),isin,ts_rows,name,ticker,start_date,end_date,start_date_ts,end_date_ts,reason_to_exclude
0,2606T9,KYG870761080,UG87076108,10X CAPITAL VENTURE ACQUISITION II A,2606T9,G87076108,VCXA,2021-10-05,2022-03-16,VCXA,...,0,AGP8696W1045,737.0,SINOVAC BIOTECH,SVA,2003-09-26,2019-02-22,2016-03-21,2019-02-22,No trades done
1,95118Z,US88025U1097,U88025U109,10X GENOMICS A,95118Z,88025U109,TXG,2019-09-12,2022-03-16,TXG,...,0,AU000000ITL3,1157.0,INTEGRATED MEDIA TECH.,IMTE,2017-08-11,2022-03-16,2017-08-11,2022-03-16,None
2,9330V3,US68247Q1022,U68247Q102,111 ADR 1:2,9330V3,68247Q102,YI,2018-09-12,2022-03-16,YI,...,0,AU0000185993,82.0,IRIS ENERGY PTY,IREN,2021-11-17,2022-03-16,2021-11-17,2022-03-16,No trades done
3,99133U,US81807M2052,U81807M205,17 ED.TGP.ADR 1:10,99133U,81807M205,YQ,2020-12-04,2022-03-16,YQ,...,0,AU0000198582,942.0,CENNTRO ELECTRIC GROUP,CENN,2018-06-20,2022-03-16,2018-06-20,2022-03-16,No trades done
4,9101KM,US68236V1044,U68236V104,180 LIFE SCIENCES,9101KM,68236V104,ATNF,2017-06-27,2022-03-16,ATNF,...,0,AU0000205205,242.0,TRITIUM DCFC,DCFC,2021-04-01,2022-03-16,2021-04-01,2022-03-16,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4073,30358V,US9898171015,U989817101,ZUMIEZ,30358V,989817101,ZUMZ,2005-05-06,2022-03-16,ZUMZ,...,0,VGG9320Z1099,40.0,VAHANNA TECH EDGE ACQUISITION I A,VHNA,2022-01-13,2022-03-11,2022-01-13,2022-03-11,None
4074,50259R,US98880R1095,U98880R109,ZW DATA ACTION TECHNOLOGIES,50259R,98880R109,CNET,2007-10-15,2022-03-16,CNET,...,0,VGG941841014,727.0,WAH FU EDUCATION GROUP,WAFU,2019-04-30,2022-03-16,2019-04-30,2022-03-16,None
4075,2568RT,US98985X1000,U98985X100,ZYMERGEN,2568RT,98985X100,ZY,2021-04-22,2022-03-16,ZY,...,0,VGG9604C1077,315.0,MEIWU TECHNOLOGY COMPANY,WNW,2020-12-15,2022-03-16,2020-12-15,2022-03-16,None
4076,98116P,US98986X1090,U98986X109,ZYNERBA PHARMACEUTICALS,98116P,98986X109,ZYNE,2015-08-05,2022-03-16,ZYNE,...,0,VGG9892K1003,1142.0,ZK INTERNATIONAL GROUP,ZKIN,2017-09-01,2022-03-16,2017-09-01,2022-03-16,None


#### Show amounts that are excluded

In [19]:
data.groupby(['reason_to_exclude']).count()[['ISIN CODE', 'Type']]

,ISIN CODE,Type
reason_to_exclude,,
Missing ISIN,1,1
NA ticker,158,158
No timeseries data,1,1
No trades done,896,896
Non-unique-ticker,93,93
None,2929,2929


In [20]:
data.groupby(['reason_to_exclude'])['trade_count'].sum()

reason_to_exclude
Missing ISIN               17
NA ticker                 790
No timeseries data       1346
No trades done              0
Non-unique-ticker       30132
None                  1115758
Name: trade_count, dtype: int64

#### Show which ones are excluded:
Also stores to csv for later use

In [21]:
#scraping_summary = data[data['reason_to_exclude']!='None']
scraping_summary = data
scraping_summary = scraping_summary[[#'Type'
                      'ISIN CODE'
                     #, 'LOC OFF. CODE'
                     , 'NAME'
                     #, 'DATASTREAM CODE'
                     #, 'CUSIP'
                     , 'TICKER SYMBOL'
                     #, 'BASE OR ST DATE'
                     #, 'DATE/TIME (DS End Date)'
                     #, 'tickers'
                     , 'trade_count'
                     #, 'min_filing_date'
                     #, 'max_filing_date'
                     #, 'min_trade_date'
                     #, 'max_trade_date'
                     , 'n_distinct_traders'
                     , 'n_distinct_trade_types'
                     #, 'P - Purchase (count)'
                     #, 'S - Sale (count)'
                     #, 'S - Sale+OE'
                     #, 'A - Grant (count)'
                     #, 'D - Sale to Iss (count)'
                     #, 'G - Gift (count)'
                     #, 'F - Tax (count)'
                     #, 'M - Option Ex (count)'
                     #, 'X - Option Ex (count)'
                     #, 'C - Cnv Deriv (count)'
                     #, 'W - Inherited (count)'
                     #, 'isin'
                     , 'ts_rows'
                     #, 'name'
                     #, 'ticker'
                     #, 'start_date'
                     #, 'end_date'
                     #, 'start_date_ts'
                     #, 'end_date_ts'
                     , 'reason_to_exclude'
                    ]].sort_values(by=['reason_to_exclude', 'TICKER SYMBOL', 'ISIN CODE'])
scraping_summary.to_csv(DATA_LOCATION + '/scraping_summary.csv')
scraping_summary

,ISIN CODE,NAME,TICKER SYMBOL,trade_count,n_distinct_traders,n_distinct_trade_types,ts_rows,reason_to_exclude
2862,NA,PB BANCORP DEAD - DELETE.02/05/20,PBBI,17,8,5,1509.0,Missing ISIN
662,BMG162491077,BROOKFIELD PR.PARTNERS UNT.,NA,5,4,2,432.0,NA ticker
3681,BMG889121031,TIZIANA LIFE SCIENCES,NA,5,4,2,1509.0,NA ticker
157,CA00971M1068,AKANDA,NA,5,4,2,1391.0,NA ticker
3412,CA83336J2083,SNOW LAKE RESOURCES,NA,5,4,2,704.0,NA ticker
...,...,...,...,...,...,...,...,...
4072,US98980G1022,ZSCALER,ZS,440,16,7,1129.0,None
4071,US98979H2022,ZOSANO PHARMA,ZSAN,61,19,6,1509.0,None
4073,US9898171015,ZUMIEZ,ZUMZ,495,26,7,40.0,None
4075,US98985X1000,ZYMERGEN,ZY,25,9,7,315.0,None


## Check how to read in, into the analysis notebook

### This should be done early in cell 3:

In [22]:
# Read in the summary data from "CompaniesToExclude" notebook
summary_data = pd.read_csv(DATA_LOCATION + '/scraping_summary.csv', index_col=0)
# Generate list of which companies to analyse
isins_to_use = summary_data[summary_data['reason_to_exclude'] == 'None']['ISIN CODE'].to_list()
display(summary_data)
print(f'We want to reduce to {len(isins_to_use)} isins')

,ISIN CODE,NAME,TICKER SYMBOL,trade_count,n_distinct_traders,n_distinct_trade_types,ts_rows,reason_to_exclude
2862,NaN,PB BANCORP DEAD - DELETE.02/05/20,PBBI,17,8,5,1509.0,Missing ISIN
662,BMG162491077,BROOKFIELD PR.PARTNERS UNT.,NaN,5,4,2,432.0,NA ticker
3681,BMG889121031,TIZIANA LIFE SCIENCES,NaN,5,4,2,1509.0,NA ticker
157,CA00971M1068,AKANDA,NaN,5,4,2,1391.0,NA ticker
3412,CA83336J2083,SNOW LAKE RESOURCES,NaN,5,4,2,704.0,NA ticker
...,...,...,...,...,...,...,...,...
4072,US98980G1022,ZSCALER,ZS,440,16,7,1129.0,None
4071,US98979H2022,ZOSANO PHARMA,ZSAN,61,19,6,1509.0,None
4073,US9898171015,ZUMIEZ,ZUMZ,495,26,7,40.0,None
4075,US98985X1000,ZYMERGEN,ZY,25,9,7,315.0,None


We want to reduce to 2929 isins


### This needs to be changed in other notebook! in Cell 3:

In [23]:
# Data locations
DATA_LOCATION = f'data/{NAME}/'
DATA_LOCATION_INSIDER_PROCESSED = DATA_LOCATION + 'processed/insider/'
DATA_LOCATION_RI = DATA_LOCATION + 'processed/RI_discard/'

## Not sure why we do this - maybe refactor
_ri_location = DATA_LOCATION_RI
_insider_location = DATA_LOCATION_INSIDER_PROCESSED

# Get locations to read in
file_locs_ = os.listdir(_ri_location)
print(f'Found {len(file_locs_)} possible files to analyze')
# Filter files for analysis, and append path:
file_locs = [_ri_location + f for f in file_locs_ if f[:-7] in isins_to_use]
print(f'We are left with {len(file_locs)} to analyze')


Found 4077 possible files to analyze
We are left with 2929 to analyze


### In the next cell (5):
I'm sorry, but this will have to replace the beautiful `ISINs = [rick[:-7] for rick in pickles]`

In [27]:
#ISINs = [f for f in file_locs_ if f[:-7] in isins_to_use]
#ISINs = isins_to_use
ISINs =  [f[:-7] for f in file_locs_ if f[:-7] in isins_to_use]
print(len(ISINs))

2929
